In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import os

import matplotlib.pyplot as plt

In [17]:
batting = pd.read_csv('core/Batting.csv')
pitching = pd.read_csv('core/Pitching.csv')
salaries = pd.read_csv('core/Salaries.csv')
all_star = pd.read_csv('core/AllStarFull.csv')

In [28]:
batting

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107424,zimmejo02,2019,1,DET,AL,23,2,0,0,0,...,0.0,0.0,0.0,0,2.0,0.0,0.0,0.0,0.0,0.0
107425,zimmeky01,2019,1,KCA,AL,15,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
107426,zimmery01,2019,1,WAS,NL,52,171,20,44,9,...,27.0,0.0,0.0,17,39.0,0.0,0.0,0.0,2.0,4.0
107427,zobribe01,2019,1,CHN,NL,47,150,24,39,5,...,17.0,0.0,0.0,23,24.0,0.0,1.0,0.0,2.0,6.0


In [19]:
batting.describe()

,yearID,stint,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
count,107429.000000,107429.000000,107429.000000,107429.000000,107429.000000,107429.000000,107429.000000,107429.000000,107429.000000,106673.000000,105061.000000,83888.000000,107429.000000,105329.000000,70778.000000,104612.000000,101360.000000,71325.000000,81988.000000
mean,1966.552141,1.079206,51.134098,140.454551,18.639818,36.714863,6.254261,1.260619,2.864785,16.929664,2.948135,1.192960,12.948785,20.677297,1.072664,1.063874,2.225345,1.036775,2.926404
std,39.549982,0.287394,47.015896,184.286687,28.161252,52.399701,9.667879,2.608268,6.396150,26.336778,7.649862,2.699623,20.683452,28.606481,2.731013,2.299923,4.175046,1.943212,4.698823
min,1871.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1936.000000,1.000000,12.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1976.000000,1.000000,34.000000,47.000000,4.000000,8.000000,1.000000,0.000000,0.000000,3.000000,0.000000,0.000000,2.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2000.000000,1.000000,80.000000,228.000000,27.000000,57.000000,9.000000,1.000000,2.000000,24.000000,2.000000,1.000000,18.000000,29.000000,1.000000,1.000000,3.000000,1.000000,4.000000
max,2019.000000,5.000000,165.000000,716.000000,198.000000,262.000000,67.000000,36.000000,73.000000,191.000000,138.000000,42.000000,232.000000,223.000000,120.000000,51.000000,67.000000,19.000000,36.000000


In [20]:
salaries.head()

,yearID,teamID,lgID,playerID,salary
0,1985,ATL,NL,barkele01,870000
1,1985,ATL,NL,bedrost01,550000
2,1985,ATL,NL,benedbr01,545000
3,1985,ATL,NL,campri01,633333
4,1985,ATL,NL,ceronri01,625000


In [21]:
salaries.describe()

,yearID,salary
count,26428.000000,2.642800e+04
mean,2000.878727,2.085634e+06
std,8.909314,3.455348e+06
min,1985.000000,0.000000e+00
25%,1994.000000,2.947020e+05
50%,2001.000000,5.500000e+05
75%,2009.000000,2.350000e+06
max,2016.000000,3.300000e+07


In [22]:
all_star.head()

,playerID,yearID,gameNum,gameID,teamID,lgID,GP,startingPos
0,gomezle01,1933.0,0.0,ALS193307060,NYA,AL,1,1.0
1,ferreri01,1933.0,0.0,ALS193307060,BOS,AL,1,2.0
2,gehrilo01,1933.0,0.0,ALS193307060,NYA,AL,1,3.0
3,gehrich01,1933.0,0.0,ALS193307060,DET,AL,1,4.0
4,dykesji01,1933.0,0.0,ALS193307060,CHA,AL,1,5.0


In [23]:
all_star.describe()

,yearID,gameNum,GP,startingPos
count,5374.000000,5374.000000,5375.000000,1691.000000
mean,1978.629512,0.097507,0.780465,5.125370
std,24.866550,0.394194,0.413970,2.657291
min,1933.000000,0.000000,0.000000,1.000000
25%,1959.000000,0.000000,1.000000,3.000000
50%,1979.000000,0.000000,1.000000,5.000000
75%,2001.000000,0.000000,1.000000,7.000000
max,2019.000000,2.000000,1.000000,10.000000


In [24]:
pitchers = np.unique(pitching.playerID)
pitchers = pd.DataFrame(pitchers)

In [25]:
pitchers

,0
0,aardsda01
1,aasedo01
2,abadfe01
3,abbeybe01
4,abbeych01
...,...
9840,zoldasa01
9841,zuberbi01
9842,zumayjo01
9843,zuverge01


In [33]:
pitchers.columns = ['playerID']
all_df = pd.merge(batting, pitchers, how='outer', on='playerID', indicator=True)
batting_stats = all_df[all_df['_merge'] == 'left_only']
batting_stats

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,_merge
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0,left_only
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0,left_only
2,addybo01,1873,1,PH2,NaN,10,51,12,16,1,...,1.0,1.0,2,0.0,NaN,NaN,NaN,NaN,0.0,left_only
3,addybo01,1873,2,BS1,NaN,31,152,37,54,6,...,6.0,5.0,2,1.0,NaN,NaN,NaN,NaN,0.0,left_only
4,addybo01,1874,1,HR1,NaN,50,213,25,51,9,...,4.0,2.0,1,1.0,NaN,NaN,NaN,NaN,0.0,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107413,waltodo01,2019,1,SEA,AL,7,16,2,3,0,...,0.0,1.0,3,5.0,0.0,0.0,0.0,0.0,0.0,left_only
107418,wongke01,2019,1,TBA,AL,6,14,1,3,0,...,0.0,1.0,0,5.0,0.0,0.0,0.0,0.0,0.0,left_only
107419,wongke01,2019,2,LAA,AL,1,4,1,0,0,...,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,left_only
107423,yastrmi01,2019,1,SFN,NL,107,371,64,101,22,...,2.0,4.0,32,107.0,1.0,4.0,1.0,3.0,4.0,left_only


In [34]:
batting_stats = batting_stats[batting_stats.yearID > 1979]
batting_stats

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,_merge
37114,minosmi01,1980,1,CHA,AL,2,2,0,0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
43751,mccarti01,1980,1,PHI,NL,6,5,2,1,1,...,0.0,0.0,1,0.0,0.0,0.0,1.0,0.0,1.0,left_only
43774,mccovwi01,1980,1,SFN,NL,48,113,8,23,8,...,0.0,0.0,13,23.0,2.0,1.0,0.0,3.0,3.0,left_only
45553,yastrca01,1980,1,BOS,AL,105,364,49,100,21,...,0.0,2.0,44,38.0,5.0,0.0,1.0,3.0,9.0,left_only
45554,yastrca01,1981,1,BOS,AL,91,338,36,83,14,...,0.0,1.0,49,28.0,4.0,0.0,0.0,3.0,10.0,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107413,waltodo01,2019,1,SEA,AL,7,16,2,3,0,...,0.0,1.0,3,5.0,0.0,0.0,0.0,0.0,0.0,left_only
107418,wongke01,2019,1,TBA,AL,6,14,1,3,0,...,0.0,1.0,0,5.0,0.0,0.0,0.0,0.0,0.0,left_only
107419,wongke01,2019,2,LAA,AL,1,4,1,0,0,...,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,left_only
107423,yastrmi01,2019,1,SFN,NL,107,371,64,101,22,...,2.0,4.0,32,107.0,1.0,4.0,1.0,3.0,4.0,left_only
